## Thank you for visit

- We have a loooot of data in this competition.
- However, we have limited RAM.
- We can save memory with PySpark in EDA.
- So, I'll show some simple usage with PySpark.

### If you like this notebook, upvote please 😉

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
# https://www.kaggle.com/ppsheth91/introductory-tutorial-1-pyspark-for-beginners

In [ ]:
!pip install pyspark > /dev/null

In [ ]:
import pyspark
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('h-and-m-personalized-fashion-recommendations').getOrCreate()

In [ ]:
class CFG:
    TRANSACTION_PATH = '../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
    ARTICLE_PATH = '../input/h-and-m-personalized-fashion-recommendations/articles.csv'
    CUSTOMER_PATH = '../input/h-and-m-personalized-fashion-recommendations/customers.csv'
    SAMPLE_PATH = '../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv'
    IMAGE_PATH = '../input/h-and-m-personalized-fashion-recommendations/images'

## Read Data

In [ ]:
train = spark.read.option('header','true').csv(CFG.TRANSACTION_PATH)

print(train.count())
train.show(5)

In [ ]:
article = spark.read.option('header','true').csv(CFG.ARTICLE_PATH)

print(article.count())
article.show(5)

In [ ]:
customer = spark.read.option('header','true').csv(CFG.CUSTOMER_PATH)

print(customer.count())
customer.show(5)

## Register Tables

In [ ]:
train.createOrReplaceTempView('transactions_train')
article.createOrReplaceTempView('articles')
customer.createOrReplaceTempView('customers')

### EDA with Spark SQL

In [ ]:
# simple read data

sample = spark.sql('''
select
  * 
from transactions_train 
limit 100
''')

sample.show(5)

In [ ]:
# group and get stats

grouped = spark.sql('''
select
  customer_id
  , collect_list(article_id) as article_id_list
  , avg(price) as avg_price
  , max(price) as max_price
  , min(price) as min_price
from transactions_train 
group by customer_id
limit 100
''')

# to pandas dataframe
df_grouped = grouped.toPandas()

print(df_grouped.shape)
df_grouped.head()

In [ ]:
# how to join

merged = spark.sql('''
select
  * 
from transactions_train as tt
left join customers as cs on cs.customer_id = tt.customer_id
left join articles as ar on ar.article_id = tt.article_id
limit 100
''')

print(merged.count())
print(merged.columns)


df_merged = merged.toPandas()

print(df_merged.shape)
df_merged.head()